In [2]:
import pandas as pd
import os
from tqdm.auto import tqdm

In [3]:
os.chdir('C:/Users/sontr/OneDrive/Desktop/Scl')

df = pd.read_csv('order_brush_order.csv').drop(columns='orderid').sort_values('event_time').set_index('event_time')
df.index = pd.to_datetime(df.index)

df.head()

,shopid,userid
event_time,,
2019-12-27 00:00:00,6042309,97707522
2019-12-27 00:00:00,104804492,97707522
2019-12-27 00:00:00,8715449,97707522
2019-12-27 00:00:02,190969466,170182475
2019-12-27 00:00:05,2859407,12532131


In [6]:
d = {}
for shopid in df.shopid.unique():
    d[shopid] = set()

In [7]:
for shopid in tqdm(df.shopid.unique()):
    shop_unq = df[df.shopid == shopid]
    ex = None
    for i, row in shop_unq.iterrows():
        start = i
        if start == ex:
            continue
            
        if ex != None:
            gap = start - ex
        
        ex = start
        end = start + pd.DateOffset(hours=1)
        no_buy = shop_unq[(shop_unq.index >= start) & (shop_unq.index <= end)]
        
        while len(no_buy) >= 3:
            con = len(no_buy) / no_buy['userid'].nunique()
            if con >= 3:
                users = no_buy.userid.value_counts()
                d[shopid].update(list(users[users == users.max()].index))
            if end - no_buy.index.max() < gap:
                no_buy = no_buy.iloc[:-1]
            else:
                break

In [8]:
out = {'shopid': [], 'userid': []}
for shopid in d:
    out['shopid'] += [shopid]
    if d[shopid] == set():
        out['userid'] += ['0']
    else:
        out['userid'] += ['&'.join(map(str, sorted(list(d[shopid]))))]

In [10]:
result = pd.DataFrame(out)
result.to_csv('solution.csv', index=False)